## Problem Statment

You have been provided a dataset that consists of echo-location clicks of two types of whales, namely, Gervais and Cuviers. In this assignment, your task is to classify the different types whales using Gradient Boosting with the help of the XGBoost library. You are expected to fill in functions that would complete this task. We use XGBoost here instead of GradientBoostedTrees in Spark because XGBoost running on a single machine is much faster than Spark running on 10 machines.

The data files were preprocessed on PySpark (10 nodes) cluster. The code for the same can be found in Data_Processing_Whales.ipynb. The preprocessed data is a numpy array with `4175` rows (for the 10mb file) with following columns (zero-indexed):
* Col 0-9: projections on first 10 eigen vectors
* Col 10: rmse
* Col 11: peak2peak
* Col 12: label (`0 if row.species==u'Gervais' else 1`)

You can also refer to XGBoost_Whales.ipynb under for more details on the XGBoost Analysis before you attempt this assignment.

Both Data_Processing_Whales.ipynb and XGBoost_Whales.ipynb can be found under XGBoost directory that was uploaded in edX as a part of "Notebooks for weeks 7 & 8". 

## XGBoost - Theory

A brief overview of gradient boosting in XGBoost can be found here:

* http://xgboost.readthedocs.io/en/latest/model.html
* http://xgboost.readthedocs.io/en/latest/python/python_intro.html



Use the XGBoost API for training and predicting scores: 

* http://xgboost.readthedocs.io/en/latest/python/python_api.html

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)

## Notebook Setup

### Importing Required Libraries

In [1]:
%matplotlib inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random

### Loading Data

In [2]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost
* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations

[Full description of options](https://xgboost.readthedocs.io/en/latest//parameter.html)

In [3]:
#You can change this cell if you wish to, but you aren't expected to
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'

    plst = param.items()
    return plst

## Exercises

### Computing the score ranges

The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays min_scr, max_scr which are calculated as follows:

1. **min_scr**: mean - (3 $\times$ std)
2. **max_scr**: mean + (3 $\times$ std)

Here the input margin scores, represents the processed XGBoost margin scores obtained from the <font color="blue">bootstrap_pred</font> function. Each row represents the various scores for a specific example in an iteration and your <font color="blue">calc_stats</font> function is supposed to compute the **min_scr** and **max_scr** as defined for each example. So in the example below, we take a scenario where we have 3 examples which have 4 values each (From 4 bootstrap iterations).


**<font color="magenta" size=2>Example Input</font>**
``` python
[[-0.22 -0.19 -0.17 -0.13][-0.1 -0.05 0.02 0.10][0.03 0.11 0.12 0.15]]
```

Output: min_scr (numpy array), max_scr (numpy array)

**<font color="blue" size=2>Example Output</font>**
``` python
(array([-0.28 -0.23 -0.03]),
 array([-0.08  0.22  0.24]))
```

**Note**: Ensure you round the values in the numpy arrays to two decimal places

In [4]:
def calc_stats(margin_scores):
    
    min_score, max_score = [], []
    for score in margin_scores:
        min_score.append(round(np.mean(score) - 3 * np.std(score), 2))
        max_score.append(round(np.mean(score) + 3 * np.std(score), 2))
    return np.array(min_score), np.array(max_score)


In [5]:
margin_score = np.array([[-0.22, -0.19, -0.17, -0.13], [-0.1, -0.05, 0.02, 0.10], [0.03, 0.11, 0.12, 0.15]])
min_score, max_score = calc_stats(margin_score)
assert type(min_score) == np.ndarray, 'Incorrect Return type'
assert type(max_score) == np.ndarray, 'Incorrect Return type'

In [6]:
assert (min_score == np.array([-0.28, -0.23, -0.03])).all(), "Incorrect return value"

In [7]:
assert (max_score == np.array([-0.08,  0.22,  0.24])).all(), "Incorrect return value"

In [8]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [9]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


### Calculating predictions

Based on the ranges for each of the examples, i.e, (min_scr, max_scr), we can predict whether it's a Gervais or a Cuvier. Since all our scores will be between -1 and +1, we use 0 as the margin line. All examples which are on the left side of the margin, can be classified as Cuvier's and all which are on the right side can be classified as Gervais. However, since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)

**<font color="magenta" size=2>Example Input</font>**
``` python
min_scr (numpy array) = [-0.78 -0.68 -0.6 -0.53 -0.47 -0.42 -0.32 -0.21 -0.07 0.22]

max_scr (numpy array) = [-0.49 -0.39 -0.33 -0.25 -0.2 -0.11 -0.04 0.1 0.3 0.51]
```
Output: pred (numpy array of predictions)

**<font color="blue" size=2>Example Output</font>**
``` python
[-1 -1 -1 -1 -1 -1 -1  0  0  1]
```

In [10]:
def predict(min_scr, max_scr):
    predict_list = []

    for i in range(len(min_s)):
        if min_s[i] < 0 and max_s[i] < 0:
            predict_list.append(-1)
        elif min_s[i] > 0 and max_s[i] > 0:
            predict_list.append(1)
        else:
            predict_list.append(0)
            
    return np.array(predict_list)
    

In [11]:
max_s = np.array([-0.49, -0.39, -0.33, -0.25, -0.2, -0.11, -0.04, 0.1, 0.3, 0.51])
min_s = np.array([-0.78, -0.68, -0.6, -0.53, -0.47, -0.42, -0.32, -0.21, -0.07, 0.22])
pred = predict(min_s, max_s)
true_pred = np.array([-1, -1, -1, -1, -1, -1, -1, 0, 0, 1])

In [12]:
assert type(pred) == np.ndarray, 'Incorrect return type'

In [13]:
assert (pred == true_pred).all(), 'Incorrect return value'

### Calculating scores

The function <font color="blue">bootstrap_pred</font> takes as input:

1. **Training set**
1. **Test set**
1. **n_bootstrap** Number of bootstrap samples that run XGBoost and trains one part of the sample set.
1. **minR, maxR** two numbers such that $0 < minR < maxR < 1$ that define the fractions of the `n_bootstrap` scores that define the range.
1. **bootstrap_size** - Number of bootstrap samples on which you will run XGBoost.
1. **num_round** - Number of iterations for running xgboost
1. **plst** - Parameter List

The output should be a confidence interval for each example in the test set. Together with a prediction that is `Gervais / Cuviers / Unsure`. The prediction `unsure` is to be output if the confidence interval contains the point 0.
After generating the confidence intervals, the function <font color="blue">predict</font> can be used to make predictions.

**Procedure**

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. Ensure you set **output_margin=True** to get raw untransformed output scores.
* Normalize them by dividing them with the normalizing factor as max(scores) - min(scores) and round these values to a precision of two decimal places

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile (sort for each example if necessary)
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      
**Note**: You can experiment by changing **n_bootstraps**, but it takes about 200 iterations to get consistent values.

In [46]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR, maxR, bootstrap_size, \
                   num_round=100, plst=xgboost_plst()):

    dtest = xgb.DMatrix(X_test,label=y_test)
    margin_scores = []
    n_bootstrap = 100
    n_samples = X_train.shape[0]
    lbound,hbound = int(n_bootstrap*minR),int(n_bootstrap*maxR)
    
    for i in range(n_bootstrap):
        # Sample boostrap_size indices from the training set with replacement
        random_idx = np.random.randint(0,n_samples,bootstrap_size)

        # Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
        X_sample,y_sample = X_train[random_idx],y_train[random_idx]
        dtrain = xgb.DMatrix(X_sample,label=y_sample)

        # Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
        evallist = [(dtrain, 'train'), (dtest, 'eval')]

        # Train the model using the XGBoost train API and make score predictions 
        model = xgb.train(params=plst,dtrain=dtrain,num_boost_round=num_round,evals=evallist,verbose_eval=False)
        pred = model.predict(dtest, ntree_limit=model.best_ntree_limit, output_margin=True)

        # Normalize them by dividing them with the normalizing factor as 
        # max(scores) - min(scores) and round these values to a precision of two decimal places
        scores = (pred/(max(pred)-min(pred))).round(2)

        margin_scores.append(np.sort(scores))
    
    margin_scores = np.array(margin_scores).T
    
    margin_scores = np.sort(margin_scores, axis=1)
    
    # For each individual example, remove scores below the minRth percentile 
    # and greater than maxRth percentile (sort for each example if necessary)
    margin_scores = margin_scores[:,lbound:hbound]
    
    # Call the calc_stats function to compute min_scr and max_scr 
    # with the filtered margin scores as parameter
    min_scr, max_scr = calc_stats(margin_scores)

    return min_scr, max_scr

In [44]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

#### Tests

How we test the function:
1. We have computed the average mid-point of the range of values and verify that this midpoint is present in the range computed by your function
2. We check that the length of the interval(max_scr-min_scr) is not more than twice the average length of the interval

In [45]:
sample_indices = np.load('Data/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/vis_midpt.npy')
avg_length = np.load('Data/vis_avg_length.npy')
min_scr, max_scr, pred = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

[[-0.64 -0.62 -0.61 -0.6  -0.6  -0.6  -0.59 -0.59 -0.59 -0.59 -0.58 -0.58
  -0.58 -0.58 -0.58 -0.58 -0.57 -0.57 -0.57 -0.57 -0.57 -0.57 -0.57 -0.57
  -0.57 -0.56 -0.56 -0.56 -0.56 -0.56 -0.56 -0.56 -0.56 -0.55 -0.55 -0.55
  -0.55 -0.55 -0.55 -0.55 -0.55 -0.55 -0.54 -0.54 -0.54 -0.54 -0.54 -0.54
  -0.54 -0.54 -0.54 -0.53 -0.53 -0.53 -0.53 -0.53 -0.53 -0.53 -0.53 -0.53
  -0.52 -0.52 -0.52 -0.52 -0.52 -0.52 -0.52 -0.52 -0.52 -0.52 -0.52 -0.51
  -0.51 -0.51 -0.51 -0.51 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  -0.5  -0.49
  -0.49 -0.49 -0.49 -0.48 -0.48 -0.47 -0.47 -0.47 -0.47 -0.47 -0.47 -0.46
  -0.46 -0.44 -0.42 -0.4 ]
 [-0.56 -0.55 -0.55 -0.54 -0.52 -0.51 -0.51 -0.5  -0.5  -0.49 -0.49 -0.49
  -0.49 -0.48 -0.47 -0.47 -0.47 -0.47 -0.46 -0.46 -0.46 -0.45 -0.45 -0.45
  -0.45 -0.45 -0.44 -0.44 -0.43 -0.43 -0.43 -0.42 -0.41 -0.41 -0.4  -0.4
  -0.4  -0.4  -0.4  -0.39 -0.38 -0.38 -0.38 -0.38 -0.38 -0.37 -0.37 -0.37
  -0.37 -0.37 -0.37 -0.36 -0.36 -0.36 -0.35 -0.35 -0.35 -0.35 -0.35 -0.35
  -0.34 -0.3

In [40]:
assert sum(min_scr <= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [41]:
assert sum(max_scr >= midpt) >= (0.7 * len(sample_indices)), "Incorrect range (mean - 3*std) to (mean + 3*std)"

In [42]:
assert sum(length < 2*avg_length) >= (0.7 * len(sample_indices)), "Incorrect length of range (mean - 3*std) to (mean + 3*std)"

In [ ]:
# Hidden Test Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests Here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#


In [ ]:
# Hidden Tests here
#
# AUTOGRADER TEST - DO NOT REMOVE
#
